In [1]:
def CopyBillingData(): 
  print("LakeName:", lakeName)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('accounts', 'billing')
  print("Reading data since", LastSyncDate)  
  
  print("Load Billing Accounts data")
  PrepareDataBronzeWithoutInstance (
    lakeName = lakeName,
    entity = BronzeTable.Accounts_Billing,
    entityName = 'tmpAccounts',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
      ba.AccountId,
      ba.Culture,
      ba.DynamicsGuid,
      ba.AccountName,
      ba.Status,
      ba.IsTest,
      ba.DataPurgedCompleted,
      ba.CreationDate,
      ba.ModificationDate,
      ba.OldAccountId,
      current_timestamp() as SilverModifiedUtc
    from tmpAccounts ba
""")  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into silver.Accounts as t
    using SourceData as s
      on t.BillingAccountId = s.AccountId or t.accountid = s.DynamicsGuid
    when matched then update set 
          BillingAccountId = s.AccountId
          , Culture = s.Culture
          , AccountName = s.AccountName
          , Status = s.Status
          , IsTest = s.IsTest
          , DataPurgedCompleted = s.DataPurgedCompleted
          , CreationDate = s.CreationDate
          , ModificationDate = s.ModificationDate
          , OldAccountId = s.OldAccountId
          , AccountId = S.DynamicsGuid
          , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( AccountId, AccountNumber, ModifiedOn, IndustryCode, BillingAccountId, Culture, AccountName, Status, IsTest, DataPurgedCompleted, CreationDate, ModificationDate, OldAccountId, SilverModifiedUtc)
      values ( DynamicsGuid, null, null, null, AccountId, Culture, AccountName, Status, IsTest, DataPurgedCompleted, CreationDate, ModificationDate, OldAccountId, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"ModificationDate": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'accounts', 'billing')

In [2]:
# PrepareData (
#     fileSystem = 'bronze',
#     dataSource = 'cab/operational',
#     instanceName = 'att', 
#     tableName = 'companies_client', 
#     pkColumns = 'CompanyId', 
#     modificationTimeColumn = 'ModifiedUtc',
#     startDateStr = '2000-03-01T00:00:00.0',
#     entityName = 'Companies'
#   )